# DM-31825 ci_imsim table check


In [1]:
import lsst.daf.butler as dafButler
import pandas as pd
pd.set_option('display.max_columns', None)

butler_ci = dafButler.Butler('/project/yusra/dp0tables/DM-31825/ci_imsim/DATA',
                              collections=['LSSTCam-imSim/runs/ci_imsim'])
butler = dafButler.Butler('/repo/dc2', collections=['2.2i/runs/test-med-1/w_2021_36/DM-31699'])

##  The Source Table


* **dimensions:** visit
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/Source.yaml
* **what**: No jointcal/FGCM for DC2 because it has a perfect reference catalog. Therefore single-frame processing is not iterative and you can assume that this is the FINAL Source Table. (This is not true for HSC, where our SourceTable is the preliminary one used for input into glocal calibration tasks)
* **primary key**:`SourceId` 
* **rows**: Contain sky sources. Be aware of flags `sky_source` and `detect_isPrimary`

**DM-31825 Notes:** Because of this table's dual purpose as science output and fgcm/jointcal input, duplicates appear for ap12Flux and ap17Flux. This will be resolved via formation of a preliminary source table to use as input for fgcm/jointcal.


In [2]:
df = butler_ci.get('sourceTable_visit', visit=212071)
print("AFTER")
print("The Source table is %s rows and %s columns" % (df.shape))
df.tail()

AFTER
The Source table is 1929 rows and 140 columns


,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,calibFlux,calibFluxErr,ap03Flux,ap03FluxErr,ap03Flux_flag,ap06Flux,ap06FluxErr,ap06Flux_flag,ap09Flux,ap09FluxErr,ap09Flux_flag,ap12Flux,ap12FluxErr,ap12Flux_flag,ap17Flux,ap17FluxErr,ap17Flux_flag,ap25Flux,ap25FluxErr,ap25Flux_flag,ap35Flux,ap35FluxErr,ap35Flux_flag,ap50Flux,ap50FluxErr,ap50Flux_flag,ap70Flux,ap70FluxErr,ap70Flux_flag,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,extendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,extendedness_flag,footprintArea_value,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_detected,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,band,instrument,detector,physical_filter,visit_system,visit
sourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
113854780169783173,56.768427,-36.393339,212071054,113854780169782823,207.706236,3951.956305,0.518565,0.731886,56.768427,-36.393339,2708.453041,1035.496322,94284.126800,19233.161463,False,166917.255714,38602.136446,False,212368.160003,58212.233195,False,204099.793355,78031.474826,False,324932.185672,111464.616028,False,420571.549991,163152.569760,False,601696.933186,227986.672285,False,NaN,NaN,True,NaN,NaN,True,2.642030,45.702714,1726.222774,320.926926,2.959527,5.740588,-3.555087,2.817938,2.916980,-0.016830,0.0,0.763623,False,0.0,False,8.565388e-07,-4.541389e-07,8.564234e-07,4.541801e-07,0.088886,False,False,False,False,False,False,3546.847900,1356.031616,False,5646.674215,1937.033021,False,436,False,0.999621,3.459864,59.849875,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1.008048,0.0,41.665974,False,False,False,False,False,False,False,False,False,False,False,False,False,4067.880615,False,False,False,False,False,False,False,True,False,False,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,r,LSSTCam-imSim,54,r_sim_1.4,1,212071
113854780169783174,56.733377,-36.339287,212071054,113854780169782828,1305.134742,3960.642036,0.499627,0.373421,56.733377,-36.339287,6342.525118,1032.360734,234583.527351,19566.065971,False,427832.203727,38929.751570,False,443644.495232,58372.210696,False,477951.083531,77795.187611,False,529296.560152,110862.230983,False,712393.726534,

In [3]:
# cols = butler_ci.get('sourceTable_visit.columns', visit=212071)
# sortedcols = [x for _, x in sorted(zip([c.lower() for c in cols], cols))]
# print('\n'.join(sortedcols))

##  The Object Table

* **dimensions:** tract 
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/Object.yaml
* **what**: The product of multi-band forced photometry (fixed centroid) and measurement. All 6 bands included, contains colors and shapes. 
* **primary key**: `objectId` 
* **rows**: Contain sky objects and patch overlaps. Be aware of flags `sky_source` and `detect_isPrimary`

**Notes:** All columns used in pipe_analysis and analysis_drp were reconciled with quantities in this table. 

In [4]:
butler_ci = dafButler.Butler('/project/yusra/dp0tables/DM-31825/ci_imsim/DATA',
                              collections=['LSSTCam-imSim/runs/ci_imsim'])
df = butler_ci.get('objectTable_tract', tract=0, patch=24, skymap='discrete/ci_imsim/4k')
print("%s rows and %s columns" % (df.shape))
df.tail()

10797 rows and 918 columns


column,xy_flag,parentObjectId,detect_isIsolated,detect_isPrimary,refBand,merge_peak_sky,detect_isTractInner,shape_xy,footprintArea,sky_object,coord_ra,detect_isDeblendedModelSource,refFwhm,detect_fromBlend,detect_isPatchInner,shape_yy,xErr,x,deblend_nChild,yErr,shape_xx,detect_isDeblendedSource,deblend_skipped,coord_dec,y,shape_flag,refExtendedness,skymap,tract,patch,u_ra,u_decl,u_psfFlux,u_psfFluxErr,u_free_psfFlux,u_free_psfFluxErr,u_free_psfFlux_flag,u_bdE1,u_bdE2,u_bdReB,u_bdReD,u_bdChi2,u_bdFluxB,u_bdFluxBErr,u_bdFluxD,u_bdFluxDErr,u_gaapFlux_psf,u_gaapFluxErr_psf,u_gaapFlux_0_7,u_gaapFluxErr_0_7,u_gaapFlux_1_0,u_gaapFluxErr_1_0,u_gaapFlux_optimal,u_gaapFluxErr_optimal,u_gaapFlux_optimal_flag_bigPsf,u_ixx,u_iyy,u_ixy,u_i_flag,u_ixxPSF,u_iyyPSF,u_ixyPSF,u_iPSF_flag,u_ixxRound,u_iyyRound,u_ixyRound,u_iRound_flag,u_ixxDebiasedPSF,u_iyyDebiasedPSF,u_ixyDebiasedPSF,u_iDebiasedPSF_flag,u_kronRad,u_kronFlux,u_kronFluxErr,u_calibFlux,u_calibFluxErr,u_ap03Flux,u_ap03FluxErr,u_ap03Flux_flag,u_ap06Flux,u_ap06FluxErr,u_ap06Flux_flag,u_ap09Flux,u_ap09FluxErr,u_ap09Flux_flag,u_ap12Flux,u_ap12FluxErr,u_ap12Flux_flag,u_ap17Flux,u_ap17FluxErr,u_ap17Flux_flag,u_ap25Flux,u_ap25FluxErr,u_ap25Flux_flag,u_ap35Flux,u_ap35FluxErr,u_ap35Flux_flag,u_ap50Flux,u_ap50FluxErr,u_ap50Flux_flag,u_ap70Flux,u_ap70FluxErr,u_ap70Flux_flag,u_extendedness,u_blendedness,u_fwhm,u_cModelFlux,u_cModelFluxErr,u_cModelFlux_inner,u_free_cModelFlux,u_free_cModelFluxErr,u_free_cModelFlux_inner,u_free_cModelFlux_flag,u_hsmShapeRegauss_e1,u_hsmShapeRegauss_e2,u_hsmShapeRegauss_sigma,u_hsmShapeRegauss_flag,u_inputCount,u_inputCount_flag,u_inputCount_flag_noInputs,u_psfFlux_area,u_psfFlux_flag,u_psfFlux_flag_apCorr,u_psfFlux_flag_edge,u_psfFlux_flag_noGoodPixels,u_cModel_flag,u_cModel_flag_apCorr,u_gaapFlux_flag,u_gaapFlux_flag_edge,u_gaapFlux_flag_gaussianization,u_gaapFlux_0_7_flag_bigPsf,u_gaapFlux_1_0_flag_bigPsf,u_blendedness_flag,u_pixelFlags_bad,u_pixelFlags_clipped,u_pixelFlags_clippedCenter,u_pixelFlags_cr,u_pixelFlags_crCenter,u_pixelFlags_edge,u_pixelFlags_inexact_psf,u_pixelFlags_inexact_psfCenter,u_pixelFlags_interpolated,u_pixelFlags_interpolatedCenter,u_pixelFlags_offimage,u_pixelFlags_saturated,u_pixelFlags_saturatedCenter,u_pixelFlags_sensor_edge,u_pixelFlags_sensor_edgeCenter,u_pixelFlags_suspect,u_pixelFlags_suspectCenter,u_extendedness_flag,u_calib_astrometry_used,u_calib_photometry_reserved,u_calib_photometry_used,u_calib_psf_candidate,u_calib_psf_reserved,u_calib_psf_used,u_apFlux_flag,u_apFlux_flag_apertureTruncated,u_apFlux_flag_sincCoeffsTruncated,u_calibFlux_flag,u_calibFlux_flag_apertureTruncated,u_calibFlux_flag_sincCoeffsTruncated,u_centroid_flag,u_centroid_x,u_centroid_xErr,u_centroid_y,u_centroid_yErr,u_kronFlux_flag,u_kronFlux_flag_bad_radius,u_kronFlux_flag_bad_shape,u_kronFlux_flag_bad_shape_no_psf,u_kronFlux_flag_edge,u_kronFlux_flag_no_fallback_radius,u_kronFlux_flag_no_minimum_radius,u_kronFlux_flag_small_radius,u_kronFlux_flag_used_minimum_radius,u_kronFlux_flag_used_psf_radius,i_ra,i_decl,i_psfFlux,i_psfFluxErr,i_free_psfFlux,i_free_psfFluxErr,i_free_psfFlux_flag,i_bdE1,i_bdE2,i_bdReB,i_bdReD,i_bdChi2,i_bdFluxB,i_bdFluxBErr,i_bdFluxD,i_bdFluxDErr,i_gaapFlux_psf,i_gaapFluxErr_psf,i_gaapFlux_0_7,i_gaapFluxErr_0_7,i_gaapFlux_1_0,i_gaapFluxErr_1_0,i_gaapFlux_optimal,i_gaapFluxErr_optimal,i_gaapFlux_optimal_flag_bigPsf,i_ixx,i_iyy,i_ixy,i_i_flag,i_ixxPSF,i_iyyPSF,i_ixyPSF,i_iPSF_flag,i_ixxRound,i_iyyRound,i_ixyRound,i_iRound_flag,i_ixxDebiasedPSF,i_iyyDebiasedPSF,i_ixyDebiasedPSF,i_iDebiasedPSF_flag,i_kronRad,i_kronFlux,i_kronFluxErr,i_calibFlux,i_calibFluxErr,i_ap03Flux,i_ap03FluxErr,i_ap03Flux_flag,i_ap06Flux,i_ap06FluxErr,i_ap06Flux_flag,i_ap09Flux,i_ap09FluxErr,i_ap09Flux_flag,i_ap12Flux,i_ap12FluxErr,i_ap12Flux_flag,i_ap17Flux,i_ap17FluxErr,i_ap17Flux_flag,i_ap25Flux,i_ap25FluxErr,i_ap25Flux_flag,i_ap35Flux,i_ap35FluxErr,i_ap35Flux_flag,i_ap50Flux,i_ap50FluxErr,i_ap50Flux_flag,i_ap70Flux,i_ap70FluxErr,i_ap70Flux_flag,i_extendedness,i_

In [5]:
import numpy as np
for col in df.columns:
    try:
        if np.sum(~np.isnan(df[col])) == 0:
            print(col)
    except:
        pass

In [6]:
# Current Columns
# Annoying long for a github render
# cols = butler_ci.get('objectTable.columns', tract=0, patch=24, skymap='discrete/ci_imsim/4k')
# sortedcols = [x for _, x in sorted(zip([c.lower() for c in cols], cols))]
# print('\n'.join(sortedcols))

## The Visit and CcdVisit Table

### Visit

* **dimensions:** None, it's global. 
* **specification:**  https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/postprocess.py#L1375
* **what**:  Visits that went into the survey
* **primary key**: `visitId`

**Note**: Yes, it's a little weird to have visitId and visit but we agreed that one is the primary key an the other is the human readable butler dimension. 

In [7]:
df = butler_ci.get('visitTable')
print("AFTER: %s rows and %s columns" % (df.shape))
df.tail()

AFTER: 30 rows and 12 columns


,visit,physical_filter,band,ra,decl,skyRotation,azimuth,altitude,zenithDistance,airmass,obsStart,expTime
visitId,,,,,,,,,,,,
496989,496989,i_sim_1.4,i,56.084552,-35.792788,127.588507,247.831759,44.203987,45.796013,1.396866,2023-12-31 05:29:44.668900,30.0
12466,12466,y_sim_1.4,y,57.695862,-36.116738,232.526056,246.020643,71.378338,18.621662,1.050812,2022-01-18 02:08:50.486200,30.0
37657,37657,y_sim_1.4,y,56.902700,-35.736180,187.925525,249.985394,59.895172,30.104828,1.144464,2022-02-20 00:53:03.427100,30.0
212739,212739,r_sim_1.4,r,57.926990,-37.998446,24.198747,114.819458,46.966065,43.033935,1.351895,2022-10-16 03:29:09.532900,30.0
257768,257768,r_sim_1.4,r,54.472185,-37.584370,114.345137,208.698964,81.519978,8.480022,1.010012,2022-12-24 02:31:13.315100,30.0


### CcdVisit

* **dimensions:** None, it's global. 
* **specification:**  https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/postprocess.py#L1375
* **what**: One row per detector.
* **primary key**: `ccdVisitId`


**Note** renamed to `ccdVisitTable`

In [8]:
df = butler_ci.get('ccdVisitTable')
print("AFTER %s rows and %s columns" % (df.shape))
df.tail()

AFTER 36 rows and 27 columns


,visitId,physical_filter,band,ra,decl,zenithDistance,zeroPoint,psfSigma,skyBg,skyNoise,detector,seeing,skyRotation,expMidpt,expTime,obsStart,darkTime,xSize,ySize,llcra,llcdec,ulcra,ulcdec,urcra,urcdec,lrcra,lrcdec
ccdVisitId,,,,,,,,,,,,,,,,,,,,,,,,,,,
496989138,496989,i_sim_1.4,i,56.602957,-36.407172,45.163715,31.849911,1.791840,3762.769775,73.177917,138,0.843840,127.588507,2023-12-31 05:29:44.668900,30.0,2023-12-31 05:29:29.668900,30.0,4071,3999,56.407939,-36.430034,56.577922,-36.249928,56.797834,-36.383985,56.628079,-36.564374
12466141,12466,y_sim_1.4,y,56.591140,-36.499298,19.445377,30.629988,2.593032,3166.046875,66.989166,141,1.220460,232.526056,2022-01-18 02:08:50.486200,30.0,2022-01-18 02:08:35.486200,30.0,4071,3999,56.616938,-36.342174,56.786100,-36.522796,56.565248,-36.656394,56.396340,-36.475486
37657141,37657,y_sim_1.4,y,56.453890,-36.634859,30.365074,30.633917,2.222234,2384.231934,58.272564,141,1.045960,187.925525,2022-02-20 00:53:03.427100,30.0,2022-02-20 00:52:48.427100,30.0,4071,3999,56.333973,-36.508984,56.612634,-36.540724,56.574191,-36.760591,56.294787,-36.728757
212739161,212739,r_sim_1.4,r,56.535845,-36.461842,42.230438,32.181396,1.533076,967.093994,37.738865,161,0.720421,24.198747,2022-10-16 03:29:09.532900,30.0,2022-10-16 03:28:54.532900,30.0,4071,3999,56.605108,-36.609795,56.350258,-36.514939,56.466899,-36.313945,56.721225,-36.408474
257768161,257768,r_sim_1.4,r,56.376124,-36.454569,6.652753,32.178368,1.626010,857.294556,35.637814,161,0.763983,114.345137,2022-12-24 02:31:13.315100,30.0,2022-12-24 02:30:58.315100,30.0,4071,3999,56.194322,-36.514710,56.304199,-36.307201,56.557530,-36.394198,56.448310,-36.601932


#  DIA Tables


## The Raw DIASource Table
* **dimensions**: `visit`
* **Specification:** https://github.com/lsst/ap_association/blob/master/data/DiaSource.yaml
* **what:** `diaSourceTable` is just an aggregated `goodSeeingDiff_diaSrcTable`. It has not yet been associated into Objects. Only purpose is QA. See next.

**Questions**:
* We're not going to bother ingesting this, but do you anyone will ever want to fetch all the DiaSources from a whole visit for QA without its `diaObjectId`?


## The DIASource Table

* **dimensions:** tract, patch
* **specification:** https://github.com/lsst/ap_association/blob/master/data/DiaSource.yaml
* **what**: DiaSource Table with diaObjectId for matching to the diaObject Table. 
* **primary key**: `diaSourceId`, foreign keys: (`diaObjectId`, `ccdVisitId`)

**Note**: There are arguments for keeping the DIASourceTable the similar to that in the APDB. The assumption that `diaSourceId` was a column rather than an index was made throughout `ap_association`. Instead of risking breaking that , I left this table as the only non-conformant in terms of:
* primary key as index-only
* psfFlux instead of psFlux

In [9]:
#butlerRFC = dafButler.Butler('/repo/dc2', collections=['u/yusra/dc2/DM-31825'])
df = butler_ci.get('diaSourceTable_tract', patch=24, tract=0, skymap='discrete/ci_imsim/4k')
print("AFTER: The table is %s rows and %s columns" % (df.shape))
df.tail()

AFTER: The table is 3651 rows and 68 columns


,diaSourceId,ccdVisitId,index,filterName,diaObjectId,ssObjectId,parentDiaSourceId,midPointTai,pixelId,bboxSize,ra,decl,x,y,xErr,yErr,apFlux,apFluxErr,snr,psFlux,psFluxErr,dipMeanFlux,dipMeanFluxErr,dipFluxDiff,dipFluxDiffErr,dipLength,dipAngle,dipChi2,isDipole,totFlux,totFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,pixelFlags,pixelFlags_offimage,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_saturated,pixelFlags_cr,pixelFlags_bad,pixelFlags_suspect,pixelFlags_interpolatedCenter,pixelFlags_saturatedCenter,pixelFlags_crCenter,pixelFlags_suspectCenter,centroid_flag,centroid_pos_flag,centroid_neg_flag,apFlux_flag,apFlux_flag_apertureTruncated,psfFlux_flag,psfFlux_flag_noGoodPixels,psfFlux_flag_edge,forced_PsfFlux_flag,forced_PsfFlux_flag_noGoodPixels,forced_PsfFlux_flag_edge,shape_flag,shape_flag_unweightedBad,shape_flag_unweighted,shape_flag_shift,shape_flag_maxIter,shape_flag_psf,coord_ra,coord_dec
3641,114213467384184914,212739161,24,r,6917529027641082361,0,0,59868.145249,9870221662731,36,56.705985,-36.432962,14.625721,3505.979661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,16011.522908,135.127311,NaN,NaN,NaN,0.099831,-0.002553,-0.002553,False,False,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,True,False,True,True,False,True,True,True,False,False,False,False,56.705985,-36.432962
3643,114213467384184917,212739161,27,r,6917529027641082362,0,0,59868.145249,9870222069552,14,56.538253,-36.355100,2816.796875,3761.153809,NaN,NaN,41.755852,387.761491,0.107684,-738.693952,137.301027,832.635672,NaN,-484.564676,NaN,NaN,NaN,NaN,False,10236.257430,143.651377,NaN,NaN,NaN,0.099746,-0.002551,-0.002551,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,56.538253,-36.355100
3645,114213467384184919,212739161,29,r,6917529027641082363,0,0,59868.145249,9870221243472,15,56.694686,-36.405335,373.049530,3889.908447,NaN,NaN,-799.736610,326.416331,-2.450051,-500.970516,88.680047,881.066609,NaN,-688.029494,NaN,NaN,NaN,NaN,False,-502.733933,105.202668,NaN,NaN,NaN,0.099814,-0.002553,-0.002553,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,56.694686,-36.405335
3649,114213467384184923,212739161,33,r,6917529027641082364,0,0,59868.145249,9870224637690,23,56.647294,-36.381968,1175.039795,3984.790283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,16267.693134,132.361292,NaN,NaN,NaN,0.099790,-0.002552,-0.002552,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,True,True,True,False,False,False,False,56.647294,-36.381968
3650,114213467384184924,212739161,34,r,6917529027641082365,0,0,59868.145249,9870222572501,13,56.604628,-36.366152,1857.670044,3984.355713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,-366.508859,104.992647,NaN,NaN,NaN,0.099770,-0.002552,-0.002552,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,True,True,True,False,False,False,False,56.604628,-36.366152


## The DIAObject Table

* **dimensions:** tract, patch
* **specification:** https://github.com/lsst/meas_base/blob/3bd782b489b14a3f46934e5e0dd43256f727c0fe/python/lsst/meas/base/diaCalculationPlugins.py
* **what**:  `DiaObjects` with aggregate metrics computes on associated `DiaSources`.
* **primary key**: `diaObjectId`, foreign keys: (`diaObjectId`, `ccdVisitId`)

**Questions:**
* 


In [10]:
df = butler_ci.get('goodSeeingDiff_diaObjTable', patch=24, tract=0, skymap='discrete/ci_imsim/4k')
print("AFTER: The table is %s rows and %s columns" % (df.shape))
print(df.columns)
df.tail()

AFTER: The table is 2949 rows and 3 columns
Index(['ra', 'decl', 'nDiaSources'], dtype='object')


,ra,decl,nDiaSources
diaObjectId,,,
6917529027641084801,56.532834,-36.473266,1
6917529027641084802,56.521293,-36.493632,1
6917529027641084803,56.662860,-36.351799,1
6917529027641084804,56.594790,-36.455703,1
6917529027641084805,56.684206,-36.395527,1


# ForcedSource Table



## ForcedSource

* **dimensions:** tract
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/ForcedSource.yaml
* **what**: Object centroids (from `deepCoadd_ref`) forced photometered on calexps and diffims in one table.  
* **primary key**: `forcedSourceId` foreign keys: (`objectId`, `ccdVisitId`)

In [11]:
df = butler_ci.get('forcedSourceTable_tract', tract=0, skymap='discrete/ci_imsim/4k')
print("AFTER: The table is %s rows and %s columns" % (df.shape))
df.head()

AFTER: The table is 150222 rows and 38 columns


,objectId,parentObjectId,coord_ra,coord_dec,ccdVisitId,band,psfFlux,psfFluxErr,psfFlux_flag,psfDiffFlux,psfDiffFluxErr,psfDiffFlux_flag,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,localBackground_instFlux,localBackground_instFluxErr,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_saturated,pixelFlags_cr,pixelFlags_bad,pixelFlags_suspect,pixelFlags_interpolatedCenter,pixelFlags_saturatedCenter,pixelFlags_crCenter,pixelFlags_suspectCenter,skymap,tract,patch,detect_isPrimary,detect_isTractInner,detect_isPatchInner
forcedSourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
193274616020467713,3458764513820540932,0,56.653378,-36.562364,180001013,u,-175.682399,223.827702,True,NaN,NaN,True,2.355587,False,0.000958,False,8.750544e-07,-4.189539e-07,8.738655e-07,4.181485e-07,0.263581,11.089581,False,False,False,False,False,False,False,False,False,False,discrete/ci_imsim/4k,0,24,False,True,False
193274616020467714,3458764513820540933,0,56.647152,-36.562586,180001013,u,2094.481580,244.745791,True,NaN,NaN,True,2.355587,False,0.000958,False,8.750816e-07,-4.188940e-07,8.738801e-07,4.180887e-07,-0.042703,10.907610,False,False,False,False,False,False,False,False,False,False,discrete/ci_imsim/4k,0,24,False,True,False
193274616020467715,3458764513820540934,0,56.553777,-36.562548,180001013,u,2294.366947,247.315716,True,NaN,NaN,True,2.355587,False,0.000958,False,8.754893e-07,-4.179904e-07,8.740997e-07,4.171935e-07,0.105257,11.157496,False,False,False,False,False,False,False,False,False,False,discrete/ci_imsim/4k,0,24,False,True,False
193274616020467716,3458764513820540935,0,56.641550,-36.562364,180001013,u,306.082458,227.543122,True,NaN,NaN,True,2.355587,False,0.000958,False,8.751062e-07,-4.188396e-07,8.738936e-07,4.180351e-07,0.299540,11.206782,False,False,False,False,False,False,False,False,False,False,discrete/ci_imsim/4k,0,24,False,True,False
193274616020467717,3458764513820540938,0,56.602678,-36.562577,180001013,u,-110.873781,224.047513,True,NaN,NaN,True,2.355587,False,0.000958,False,8.752761e-07,-4.184638e-07,8.739852e-07,4.176624e-07,0.361009,10.995247,False,False,False,False,False,False,False,False,False,False,discrete/ci_imsim/4k,0,24,False,True,False


## ForcedSourceOnDiaObjects Table

* **dimensions:** tract
* **specification:** https://github.com/lsst/obs_lsst/blob/master/pipelines/imsim/DRP.yaml
* **what**: diaObject centroids (from `goodSeeingDiff_diaObjTable`) forced photometered on calexps and diffims in one table.  
* **primary key**:  `forcedSourceOnDiaObjectId` with foreign keys: (`diaObjectId`, `ccdVisitId`)


**Note:**
Did not get to `detect_isTractInner` in time. I might include a placeholder. 

In [12]:
df = butler_ci.get('forcedSourceOnDiaObjectTable_tract', patch=24, tract=0, skymap='discrete/ci_imsim/4k')
print("AFTER")
print("The Source table is %s rows and %s columns" % (df.shape))
df.tail()

AFTER
The Source table is 46987 rows and 35 columns


,diaObjectId,parentObjectId,coord_ra,coord_dec,ccdVisitId,band,psfFlux,psfFluxErr,psfFlux_flag,psfDiffFlux,psfDiffFluxErr,psfDiffFlux_flag,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,localBackground_instFlux,localBackground_instFluxErr,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_saturated,pixelFlags_cr,pixelFlags_bad,pixelFlags_suspect,pixelFlags_interpolatedCenter,pixelFlags_saturatedCenter,pixelFlags_crCenter,pixelFlags_suspectCenter,skymap,tract,patch
forcedSourceOnDiaObjectId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
228426934768371903,6917529027641084800,0,56.595783,-36.455950,212739161,r,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discrete/ci_imsim/4k,0,24
228426934768371904,6917529027641084801,0,56.532834,-36.473266,212739161,r,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discrete/ci_imsim/4k,0,24
228426934768371905,6917529027641084802,0,56.521293,-36.493632,212739161,r,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discrete/ci_imsim/4k,0,24
228426934768371906,6917529027641084804,0,56.594790,-36.455703,212739161,r,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discrete/ci_imsim/4k,0,24
228426934768371907,6917529027641084805,0,56.684206,-36.395527,212739161,r,NaN,NaN,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discrete/ci_imsim/4k,0,24
